# Step By Step Pengerjaan (Deadline 3 September 2024)
- Cocokin data byPlaceData.csv dengan scrapetable_wisata.xlsx
- Mengubah konten menjadi bahasa indonesia, dengan mengambil data dari byPlaceData.csv
- Data yang tidak ada di scrapetable_wisata.xlsx langsung di drop saja di byPlaceData.csv (mengurangi storage)

# Data Gathering

In [ ]:
import pandas as pd

# dataLamaPath = 'data/new-fix-data.csv'
dataBaruPath = 'data/scrapetable_wisata.xlsx'
dataByPlace = 'data/byPlaceData.csv'

# dfOld = pd.read_csv(dataLamaPath)
dfNew = pd.read_excel(dataBaruPath)
dfByPlace = pd.read_csv(dataByPlace)

In [ ]:
dfNew.head()

In [ ]:
dfByPlace.head()

# Data Assessing

## Cek Info DF

In [ ]:
# ByPlace
print('By Place')
print(dfByPlace.info())

print()

# new
print('New')
print(dfNew.info())

## Cek Nilai Null

In [ ]:
# ByPlace
print('By Place')
print(dfByPlace.isna().sum())

print()

# new
print('New')
print(dfNew.isna().sum())

## Cek Data Duplikat

In [ ]:
# ByPlace
print('By Place')
print(dfByPlace.duplicated().sum())

print()

# new
print('New')
print(dfNew.duplicated().sum())

# Data Cleaning

## Feature Selection